In [1]:
import tkinter as tk
from tkinter import ttk
from tkinter import font
import pandas as pd
import joblib
import numpy as np
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

path=""
type_of_file=""
dataset=""
validation_size=""
random_state=""
y_parameter=""
parameters=[]
remaining=[]
models=[]

def set_path():
    global path
    path=x1.get()
    load_dataset()

def set_type():
    global type_of_file
    type_of_file=x2.get()
    
def show_head():
    global dataset
    window=tk.Tk()
    window.title("Head of Dataset")
    head=dataset.head()
    head=np.array(head)
    rows,columns=head.shape
    labels=dataset.columns.values
    for i in range(rows): #Rows
        if i==0:
            for j in range(columns):
                b = ttk.Label(window, text="{}".format(labels[j]))
                b.grid(row=i, column=j)
        for j in range(columns): #Columns
            b = ttk.Label(window, text="{}".format(float(head[i][j])))
            b.grid(row=i+1, column=j)
    ttk.Button(window,text='Close',command=window.destroy).grid(row=rows+1,column=columns-1)
    tk.mainloop()

def load_dataset():
    global path,type_of_file,dataset
    if path=="":
        pass
    else:
        dataset=pd.read_csv(path)
        
def set_validation():
    global validation_size
    validation_size=float(x3.get())
    
def set_random():
    global random_state
    random_state=int(x4.get())
    
def select_params():
    global parameters
    params = list()
    selec = list_para.curselection()
    for i in selec:
        j = list_para.get(i)
        params.append(j)
    parameters=params

def set_y_param():
    global y_parameter
    y_parameter=y_param_option.get(y_param_option.curselection())
    
def set_subtract(A,B):
    """
    A and B are lists
    Remove the contents of B from A and returns a new list
    """
    A,B=list(A),list(B)
    for i in B:
        if i in A:
            A.remove(i)
    return A
    
def get_y_param():
    global remaining
    remaining=set_subtract(dataset.columns.values,parameters)
    temp=" ".join(remaining)
    x6.set(temp)
    
def select_models():
    global models
    model = list()
    selec = list_model.curselection()
    for i in selec:
        j = list_model.get(i)
        model.append(j)
    models=model
    
def train_models(dataset,x_parameters,y_parameter,models,validation_size,random_state):
    """
    Train different classification Machine Learning algorithm
    PARAMETERS
    dataset: Pandas dataframe of the dataset
    x_parameters: list of labels use for training purpose
    y_parameter: to be predicted
    models: Models want to be used among LR, LDA, KNN, DT, NB, SVM
    validation_size: Validation size
    random_state: Random state
    """
    training_models=[]
    result=[]
    model_dict={"LR":LogisticRegression(),
                "LDA":LinearDiscriminantAnalysis(),
                "KNN":KNeighborsClassifier(),
                "DT":DecisionTreeClassifier(),
                "NB":GaussianNB(),
                "SVM":SVC()
               }
    X=dataset.loc[:,parameters]
    X=X.values
    Y=dataset.loc[:,y_parameter]
    Y=Y.values
    x_train,x_test,y_train,y_test=model_selection.train_test_split(X,Y,test_size=validation_size,random_state=random_state)
    for i in models:
        training_models.append((i,model_dict[i]))
    for name,mod in training_models:
        mod.fit(x_train,y_train)
        result.append([name,mod.score(x_test,y_test)])
    return result
    
def result():
    global dataset,parameters,y_parameter,models,validation_size,random_state
    result=train_models(dataset,parameters,y_parameter,models,validation_size,random_state)
    window=tk.Tk()
    window.title("Result")
    for i in range(len(result)):
        for j in range(2):
            ttk.Label(window,text=str(result[i][j])).grid(row=i,column=j)
    tk.mainloop()
    
def train_wrapper(window):
    window.iconify()
    train()

    
window=tk.Tk()
window.title("ML-Classifier")
window.resizable(False,False)

x1=tk.StringVar()
x2=tk.StringVar()
x3=tk.StringVar()
x4=tk.StringVar()
x5=tk.StringVar()
x6=tk.StringVar()
x7=tk.StringVar()
x7.set("LR LDA KNN DT NB SVM")


label_frame1=ttk.LabelFrame(window,height=100,width=200,text="Selecting Path and Type of file")
label_frame1.grid(row=0,column=0,columnspan=2)

ttk.Label(label_frame1,text="Enter the path of dataset").grid(row=0,column=0)
ttk.Entry(label_frame1,textvariable=x1).grid(row=0,column=1)
ttk.Button(label_frame1,text="Submit",command=lambda : set_path()).grid(row=0,column=2)

ttk.Label(label_frame1,text="Enter the type of file").grid(row=1,column=0)
ttk.Combobox(label_frame1,values=["csv","others"],textvariable=x2).grid(row=1,column=1)
ttk.Button(label_frame1,text="Submit",command=lambda:set_type()).grid(row=1,column=2)



label_frame2=ttk.LabelFrame(window,height=100,width=200,text="Opening head of dataset")
label_frame2.grid(row=1,column=0,columnspan=2)
ttk.Button(label_frame2,text="Dataset head",command=lambda : show_head()).pack()



label_frame3=ttk.LabelFrame(window,height=100,width=200,text="Validation Size and Random State")
label_frame3.grid(row=2,column=0,columnspan=2)

ttk.Label(label_frame3,text="Enter the Validation Size").grid(row=0,column=0)
ttk.Entry(label_frame3,textvariable=x3).grid(row=0,column=1)
ttk.Button(label_frame3,text="Submit",command=lambda : set_validation()).grid(row=0,column=2)

ttk.Label(label_frame3,text="Enter the Random State").grid(row=1,column=0)
ttk.Entry(label_frame3,textvariable=x4).grid(row=1,column=1)
ttk.Button(label_frame3,text="Submit",command=lambda : set_random()).grid(row=1,column=2)



# x_parameters
label_frame4=ttk.LabelFrame(window,height=100,width=200,text="Select x-Parameters to feed as input")
label_frame4.grid(row=3,column=0)

ttk.Button(label_frame4,text="Show list",command=lambda: x5.set(" ".join(dataset.columns.values))).pack()
list_para=tk.Listbox(label_frame4,listvariable=x5,selectmode=tk.MULTIPLE)
list_para.pack()
ttk.Button(label_frame4,text="Submit",command=lambda:select_params()).pack()



# y_parameters
label_frame5=ttk.LabelFrame(window,height=100,width=200,text="Select the column to be predicted")
label_frame5.grid(row=3,column=1)

ttk.Button(label_frame5,text="Show list",command=lambda : get_y_param()).pack()
y_param_option=tk.Listbox(label_frame5,listvariable=x6,selectmode=tk.SINGLE)
y_param_option.pack()
ttk.Button(label_frame5,text="submit",command=lambda : set_y_param()).pack()



# Selecting models
label_frame6=ttk.LabelFrame(window,height=100,width=200,text="Select the type of models")
label_frame6.grid(row=4,column=0,columnspan=2)

list_model = tk.Listbox(label_frame6,listvariable=x7,selectmode=tk.MULTIPLE)
list_model.pack()
ttk.Button(label_frame6, text="Select", command=lambda : select_models()).pack()

ttk.Button(window,text="Result",command=lambda : result()).grid(row=5,column=0,columnspan=2)

tk.mainloop()